In [1]:
import sys
# sys.path.append(r'E:/0code')
sys.path.append(r'/home/wyf/0code')
# sys.path.append(r'/home/wangyf226/0code')
# sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/pyml')

In [40]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
from pyml.feature_extraction.text import CountVectorizer
from pyml.linear_model.regression import LinearRegression
from pyml.neighbors.classification import KNeighborsClassifier
from pyml.metrics.regression import pearson_correlation
from pyml.model_selection import KFold
from pyml.model_selection import ShuffleSplit
from pyml.preprocessing import StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 读取数据文件

In [9]:
train = pd.read_excel('../data/train.xlsx')
test = pd.read_excel('../data/testStudent.xlsx')

In [10]:
train.dtypes # 检查有没有数据类型错误的，比如原本是int的变成str，说明里面可能有nan值等奇怪的数据

Additional_Number_of_Scoring                    int64
Average_Score                                 float64
Review_Total_Negative_Word_Counts               int64
Total_Number_of_Reviews                         int64
Review_Total_Positive_Word_Counts               int64
Total_Number_of_Reviews_Reviewer_Has_Given      int64
Tags                                           object
Reviewer_Score                                float64
dtype: object

In [21]:
train_ori_X = train.drop('Reviewer_Score', axis=1).drop('Tags', axis=1)
train_ori_Y = train['Reviewer_Score']
test_ori_X = test.drop('Tags', axis=1)

# 特征工程

In [27]:
def get_proportion_feature(df):
    """
    构造以下三个特征
    积极评论占总评论的比例
    消极评论占总评论的比例
    评论员评论占总评论的比例
    """
    df = df.copy()
    
    base_features = ['Total_Number_of_Reviews']
    gap_features = ['Review_Total_Negative_Word_Counts', 'Review_Total_Positive_Word_Counts','Total_Number_of_Reviews_Reviewer_Has_Given']
    for base_feature in base_features:
        for gap_feature in gap_features:
            df[gap_feature+'_radio_'+base_feature] = df[gap_feature]/df[base_feature]
            # 数字太小了，乘上一个10
#             df = df.drop(gap_feature, axis=1)
    return df

# 构造训练集和测试集，并归一化

In [28]:
train_X_feat = get_proportion_feature(train_ori_X)
test_X_feat = get_proportion_feature(test_ori_X)

In [29]:
train_X_feat.columns

Index(['Additional_Number_of_Scoring', 'Average_Score',
       'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given',
       'Review_Total_Negative_Word_Counts_radio_Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts_radio_Total_Number_of_Reviews',
       'Total_Number_of_Reviews_Reviewer_Has_Given_radio_Total_Number_of_Reviews'],
      dtype='object')

In [31]:
ss = StandardScaler()
train_X = ss.fit_transform(train_X_feat.values)
test_X = ss.transform(test_X_feat.values)

In [32]:
train_Y = train_ori_Y.values

# 交叉验证

In [45]:
n_splits = 3
cv = ShuffleSplit(n_splits=n_splits)
for train_indices, test_indices in cv.split(train_X):
    lr = LinearRegression(learning_rate=0.01, num_iterations=10000)
    lr.fit(train_X[train_indices], train_Y[train_indices], watch=True)
    y_pred = lr.predict(train_X[test_indices])
    print(pearson_correlation(y_pred, train_Y[test_indices]))

Cost after iteration 0: 36.634699
Cost after iteration 100: 5.704859
Cost after iteration 200: 1.577265
Cost after iteration 300: 1.021914
Cost after iteration 400: 0.945717
Cost after iteration 500: 0.934503
Cost after iteration 600: 0.932448
Cost after iteration 700: 0.931868
Cost after iteration 800: 0.931623
Cost after iteration 900: 0.931497
Cost after iteration 1000: 0.931428
Cost after iteration 1100: 0.931390
Cost after iteration 1200: 0.931369
Cost after iteration 1300: 0.931357
Cost after iteration 1400: 0.931351
Cost after iteration 1500: 0.931347
Cost after iteration 1600: 0.931345
Cost after iteration 1700: 0.931344
Cost after iteration 1800: 0.931343
Cost after iteration 1900: 0.931342
Cost after iteration 2000: 0.931342
Cost after iteration 2100: 0.931342
Cost after iteration 2200: 0.931342
Cost after iteration 2300: 0.931342
Cost after iteration 2400: 0.931342
Cost after iteration 2500: 0.931342
Cost after iteration 2600: 0.931342
Cost after iteration 2700: 0.931342
Cos

Cost after iteration 2800: 0.931977
Cost after iteration 2900: 0.931977
Cost after iteration 3000: 0.931977
Cost after iteration 3100: 0.931977
Cost after iteration 3200: 0.931977
Cost after iteration 3300: 0.931977
Cost after iteration 3400: 0.931977
Cost after iteration 3500: 0.931977
Cost after iteration 3600: 0.931977
Cost after iteration 3700: 0.931977
Cost after iteration 3800: 0.931977
Cost after iteration 3900: 0.931977
Cost after iteration 4000: 0.931977
Cost after iteration 4100: 0.931977
Cost after iteration 4200: 0.931977
Cost after iteration 4300: 0.931977
Cost after iteration 4400: 0.931977
Cost after iteration 4500: 0.931977
Cost after iteration 4600: 0.931977
Cost after iteration 4700: 0.931977
Cost after iteration 4800: 0.931977
Cost after iteration 4900: 0.931977
Cost after iteration 5000: 0.931977
Cost after iteration 5100: 0.931977
Cost after iteration 5200: 0.931977
Cost after iteration 5300: 0.931977
Cost after iteration 5400: 0.931977
Cost after iteration 5500: 0

# 训练模型写入结果

In [49]:
lr = LinearRegression(learning_rate=0.01, num_iterations=3000)
lr.fit(train_X, train_Y, watch=True)

Cost after iteration 0: 36.645943
Cost after iteration 100: 5.705109
Cost after iteration 200: 1.577082
Cost after iteration 300: 1.022013
Cost after iteration 400: 0.945920
Cost after iteration 500: 0.934729
Cost after iteration 600: 0.932671
Cost after iteration 700: 0.932082
Cost after iteration 800: 0.931828
Cost after iteration 900: 0.931695
Cost after iteration 1000: 0.931622
Cost after iteration 1100: 0.931581
Cost after iteration 1200: 0.931558
Cost after iteration 1300: 0.931545
Cost after iteration 1400: 0.931537
Cost after iteration 1500: 0.931533
Cost after iteration 1600: 0.931530
Cost after iteration 1700: 0.931529
Cost after iteration 1800: 0.931528
Cost after iteration 1900: 0.931528
Cost after iteration 2000: 0.931527
Cost after iteration 2100: 0.931527
Cost after iteration 2200: 0.931527
Cost after iteration 2300: 0.931527
Cost after iteration 2400: 0.931527
Cost after iteration 2500: 0.931527
Cost after iteration 2600: 0.931527
Cost after iteration 2700: 0.931527
Cos

In [50]:
y_pred = lr.predict(test_X)
sub = pd.DataFrame(y_pred)
sub.to_csv('../results/'+'LinearRegression-0.01-3000-'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)